# OrbitPy propagation example (J2 and SGP4 propagators)

Specifies a function with well-defined input (`PropagationRequest`) and output (`PropagationResponse`) using OrbitPy's implementation.

In [1]:
import warnings

import tempfile
import os, shutil

script_directory = os.path.dirname(os.path.abspath("__file__"))
temp_dir = os.path.join(script_directory, "temp")
os.makedirs(temp_dir, exist_ok=True)


import json
from datetime import datetime, timedelta, timezone


import pandas as pd


from pydantic import AwareDatetime

from orbitpy.util import OrbitState as OrbitPy_OrbitState, Spacecraft as OrbitPy_Spacecraft
from orbitpy.propagator import J2AnalyticalPropagator as OrbitPy_J2AnalyticalPropagator, SGP4Propagator as OrbitPy_SGP4Propagator

from eose.orbits import GeneralPerturbationsOrbitState, Propagator
from eose.satellites import Satellite
from eose.utils import CartesianReferenceFrame, PlanetaryCoordinateReferenceSystem, Quaternion
from eose.geometry import Position

from eose.propagation import (
    PropagationSample,
    PropagationRecord,
    PropagationRequest,
    PropagationResponse,
)

from eose.instruments import CircularGeometry, RectangularGeometry, BasicSensor

pd.set_option('display.max_rows', None)

### Define mission parameters

In [2]:
# define the orbit and the instrument
iss_omm_str = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss_omm = json.loads(iss_omm_str)[0]

basic_sensor = BasicSensor(   id="Atom",
                        mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = RectangularGeometry(angle_height=60.0, angle_width=30), #CircularGeometry(diameter=60.0)
                        orientation= list([0, 0.258819, 0, 0.9659258]), # +30 deg roll about x-axis (roll)
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

satellites=[
        Satellite(
            id="ISS",
            orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm),
            payloads=[
                basic_sensor
            ]
        )
    ]

mission_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
mission_duration = timedelta(days=1)
propagate_time_step = timedelta(minutes=1)

### Run propagation calculations with OrbitPy

J2 or SGP4 propagators can be chosen.


In [3]:
def propagate_orbitpy(request: PropagationRequest) -> PropagationResponse:

    if request.frame != CartesianReferenceFrame.ICRF:
        raise ValueError(f"OrbitPy supports only CartesianReferenceFrame.ICRF and does not support {request.frame}")
    
    #### Enumerate and convert from EOSE-API satellites to OrbitPy satellite objects. ####
    # (Enumeration generates distinct orbit-instrument pairs for satellites equipped with multiple instruments.)
    OrbitPy_Satellites = []
    for satellite in request.satellites:
                
        tle = satellite.orbit.to_tle()

        orbit_state = OrbitPy_OrbitState.from_dict({"tle": {
                                                    "tle_line0": "Unknown",
                                                    "tle_line1": tle[0],
                                                    "tle_line2": tle[1]
                                                    }})
        
        sat = OrbitPy_Spacecraft(_id = satellite.id,
                            orbitState = orbit_state  #satellite-bus and instrument attributes are ignored since they are not relevant to propagation
                        )
        OrbitPy_Satellites.append(sat)
    
    #### run propagation and coverage with OrbitPy ####
    step_size_s = request.time_step.total_seconds()
    if request.propagator != Propagator.J2:
        propagator = OrbitPy_J2AnalyticalPropagator.from_dict({"@type": "J2 ANALYTICAL PROPAGATOR", "stepSize": step_size_s})
    elif request.propagator != Propagator.SGP4:
        propagator = OrbitPy_SGP4Propagator.from_dict({"@type": "SGP4 PROPAGATOR", "stepSize": step_size_s})
    else:
        raise RuntimeError("OrbitPy only supports J2 and SGP4 propagators.")
            
    start_date_dict = {"@type": "GREGORIAN_UT1", "year": request.start.year, "month": request.start.month, "day": request.start.day, "hour": request.start.hour, "minute": request.start.minute, "second": request.start.second + request.start.microsecond / 1000000.0}
    start_date = OrbitPy_OrbitState.date_from_dict(start_date_dict) # assumed that the time scale is UT1
    duration = request.duration.total_seconds() / 86400.0
    
    satellite_records = []
    for orbitpy_sat in OrbitPy_Satellites:

        # run propagation with OrbitPy
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False,dir=temp_dir) as state_cart_file: # store satellite states in a temporary file.
            propagator.execute(orbitpy_sat, start_date, state_cart_file.name, None, duration)

        # transform the OrbitPy propagation results to PropagationResponse object
        orbitpy_states_df = pd.read_csv(state_cart_file.name, skiprows=4)

        # iterate over the dataframe and form PropagationSamples
        propagation_samples = []
        for index, row in orbitpy_states_df.iterrows():
            propagation_samples.append(PropagationSample(
                                                time = request.start + timedelta(seconds=step_size_s*row["time index"]),
                                                position = [row["x [km]"]*1e3, row["y [km]"]*1e3, row["z [km]"]*1e3],
                                                velocity = [row["vx [km/s]"]*1e3, row["vy [km/s]"]*1e3, row["vz [km/s]"]*1e3]
                                    ))
                                        
        satellite_records.append(PropagationRecord(
                                    satellite_id = orbitpy_sat._id,
                                    samples = propagation_samples
                                ))

    
    # delete the temporary directory
    shutil.rmtree(temp_dir)
    
    return PropagationResponse(
            **request.model_dump(exclude="satellite_records"),
            satellite_records=satellite_records
        )

In [ ]:
request = PropagationRequest(
    satellites=satellites,
    start=mission_start,
    duration=mission_duration,
    frame=CartesianReferenceFrame.ICRF,
    propagator=Propagator.J2
)

#display(request.model_dump_json())

propagate_response = propagate_orbitpy(request)

#display(propagate_response.model_dump_json())

propagate_data = propagate_response.as_dataframe()

display(propagate_data)